In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = r'../kaggle/working/train' 
Validation_path = r'../kaggle/working/validation' 

train_path_normal=r'../kaggle/working/train/normal' 
train_path_bacteria=r'../kaggle/working/train/bacteria' 
train_path_virus=r'../kaggle/working/train/virus' 

Validation_path_normal = r'../kaggle/working/validation/normal' 
Validation_path_bacteria = r'../kaggle/working/validation/bacteria'
Validation_path_virus = r'../kaggle/working/validation/virus'

if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(Validation_path):
    os.makedirs(Validation_path)    
if not os.path.exists(train_path_normal):
    os.makedirs(train_path_normal)
if not os.path.exists(train_path_bacteria):
    os.makedirs(train_path_bacteria)
if not os.path.exists(train_path_virus):
    os.makedirs(train_path_virus)
if not os.path.exists(Validation_path_normal):
    os.makedirs(Validation_path_normal)
if not os.path.exists(Validation_path_bacteria):
    os.makedirs(Validation_path_bacteria)
if not os.path.exists(Validation_path_virus):
    os.makedirs(Validation_path_virus)

In [ ]:
from shutil import copy2
import cv2
data=pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')
Normal=[]
Virus=[]
Bacteria=[]
for i in range(data['Label'].size):
    if data['Dataset_type'][i]=='TRAIN':
        if data['Label'][i]=='Normal':
            copy2('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'\
                     +data['X_ray_image_name'][i],r'../kaggle/working/train/normal/'+data['X_ray_image_name'][i])
        if data['Label'][i]=='Pnemonia':
            if data['Label_1_Virus_category'][i]=='Virus':
                copy2('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'\
                     +data['X_ray_image_name'][i],r'../kaggle/working/train/virus/'+data['X_ray_image_name'][i])
            if data['Label_1_Virus_category'][i]=='bacteria':
                copy2('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'\
                     +data['X_ray_image_name'][i],r'../kaggle/working/train/bacteria/'+data['X_ray_image_name'][i])
    if data['Dataset_type'][i]=='TEST':
        if data['Label'][i]=='Normal':
            copy2('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'\
                     +data['X_ray_image_name'][i],r'../kaggle/working/validation/normal/'+data['X_ray_image_name'][i])
        if data['Label'][i]=='Pnemonia':
            if data['Label_1_Virus_category'][i]=='Virus':
                copy2('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'\
                     +data['X_ray_image_name'][i],r'../kaggle/working/validation/virus/'+data['X_ray_image_name'][i])
            if data['Label_1_Virus_category'][i]=='bacteria':
                copy2('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'\
                     +data['X_ray_image_name'][i],r'../kaggle/working/validation/bacteria/'+data['X_ray_image_name'][i])

In [ ]:
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import Model
#################################################################
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (500, 500, 3), 
                                include_top = False, 
                                weights='imagenet')
for layer in pre_trained_model.layers:
    layer.trainable = False    
pre_trained_model.summary()
last_layer = pre_trained_model.get_layer('mixed10')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
#################################################################
from tensorflow.keras.applications import Xception
pre_trained_model=Xception(include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(300, 300,3), pooling='max', classes=None,
    classifier_activation='softmax')
for layer in pre_trained_model.layers:
      layer.trainable = False
pre_trained_model.summary() 
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
#################################################################
from tensorflow.keras.applications import VGG19
pre_trained_model=VGG19(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(500, 500, 3), pooling='max', classes=None,
    classifier_activation='softmax')
pre_trained_model.summary()
for layer in pre_trained_model.layers:
      layer.trainable = False
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
#################################################################
from tensorflow.keras.applications import ResNet152V2
pre_trained_model=ResNet152V2(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(500, 500, 3), pooling='max', classes=None,
    classifier_activation='softmax')
pre_trained_model.summary()
for layer in pre_trained_model.layers:
      layer.trainable = False
pre_trained_model.summary()
last_layer = pre_trained_model.get_layer('post_relu')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(3, activation='softmax').apply(x)           
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(lr=0.0001),loss = 'categorical_crossentropy',metrics = ['CategoricalAccuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 0,
                                   width_shift_range = 0.2,
                                   fill_mode='reflect',
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   channel_shift_range=150.0/255.,
                                   vertical_flip=True,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator = train_datagen.flow_from_directory(train_path,
                                                    batch_size = 20,
                                                    class_mode = 'categorical', 
                                                    target_size = (500, 500))
validation_generator = test_datagen.flow_from_directory(Validation_path,
                                                          batch_size  = 20,
                                                          class_mode  = 'categorical', 
                                                          target_size = (500, 500))


In [ ]:
history = model.fit(train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 18,
            validation_steps = 20,
            verbose = 2)